In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

  Using cached accelerate-0.29.3-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0
Found existing installation: transformers 4.39.1
Uninstalling transformers-4.39.1:
  Successfully uninstalled transformers-4.39.1
Found existing installation: accelerate 0.29.3
Uninstalling accelerate-0.29.3:
  Successfully uninstalled accelerate-0.29.3


In [ ]:
# Import necessary modules
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset, load_metric, concatenate_datasets
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

nltk.download("punkt")

c:\Users\priks\anaconda3\envs\text-summarizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\priks\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
token = "User Access Tokens"
import subprocess
subprocess.run(["huggingface-cli", "login", "--token", token], check=True)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Define the checkpoint or model identifier for the BART model
model_ckpt = "google-t5/t5-small"

# Initialize a tokenizer using the pretrained BART model's tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Load the pretrained BART model for sequence-to-sequence tasks
# and move it to a specified device (e.g., GPU if available)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)


In [4]:
# load the dataset, using samsum dataset from hugging face

from datasets import load_dataset

dataset_name = "samsum"
dataset = load_dataset(dataset_name)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [6]:
print(f"text: \n{dataset['train'][1]['dialogue']}, \n\nsummary: \n{dataset['train'][1]['summary']}")

text: 
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great, 

summary: 
Olivia and Olivier are voting for liberals in this election. 


In [7]:
# Split the dataset into training, testing, and validation sets
train_dataset = dataset['train']
test_dataset = dataset['test']
validation_dataset = dataset['validation']

# Print information about the datasets
print("Training dataset:")
print(train_dataset)
print("Testing dataset:")
print(test_dataset)
print("Validation dataset:")
print(validation_dataset)

Training dataset:
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})
Testing dataset:
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})
Validation dataset:
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})


In [9]:
prefix = "summarize: "

def convert_examples_to_features(example_batch):
    # Tokenize the dialogue from the example batch
    inputs = [prefix + doc for doc in example_batch['dialogue']]
    model_inputs = tokenizer(
        inputs,                     # Extract dialogue from the example batch
        max_length=1024,             # Set maximum sequence length for input
        truncation=True             # Truncate sequences that exceed max_length
    )

    # Tokenize the summary from the example batch as target
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example_batch['summary'],  # Extract summary from the example batch
            max_length=128,             # Set maximum sequence length for summary
            truncation=True            # Truncate sequences that exceed max_length
        )

    
    model_inputs["labels"] = labels["input_ids"]
    # Return a dictionary containing input and target encodings
    return model_inputs


In [10]:
dataset_samsum_pt = dataset.map(convert_examples_to_features, batched = True) # batched=True to process multiple elements of the dataset at once

In [11]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [12]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [13]:
dataset_samsum_pt["train"][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.',
 'input_ids': [21603,
  10,
  21542,
  10,
  27,
  13635,
  5081,
  5,
  531,
  25,
  241,
  128,
  58,
  16637,
  10,
  10625,
  55,
  21542,
  10,
  27,
  31,
  195,
  830,
  25,
  5721,
  3,
  10,
  18,
  61,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [21542, 13635, 5081, 11, 56, 830, 16637, 128, 5721, 5, 1]}

In [14]:
# Training

from transformers import DataCollatorForSeq2Seq

# create a batch of examples using DataCollatorForSeq2Seq. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
     

In [15]:
from transformers import Trainer, TrainingArguments

# Adjusted parameters for better GPU utilization and checkpoint saving
training_args = TrainingArguments(
    output_dir='t5-small-samsum',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=64,             # total number of training epochs
    per_device_train_batch_size=8,   # batch size
    per_device_eval_batch_size=32,    # batch size for evaluation
    warmup_steps=500,
    weight_decay=0.01,
    save_total_limit=3,               # Save the last 5 checkpoints
    gradient_accumulation_steps=4,    # Gradient accumulation
    logging_dir='./logs',             # Directory for logs
    save_strategy="epoch",            # Save checkpoint at the end of each epoch
    load_best_model_at_end=True,      # Load the best model based on early stopping
    metric_for_best_model="eval_loss",# Metric to use for early stopping
    greater_is_better=False,          # Whether the metric should be minimized or maximized
    fp16=True,                        # Mixed precision training
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    tokenizer=tokenizer,                 # the tokenizer used to preprocess the data
    data_collator=seq2seq_data_collator, # data collator used for batching and padding
    train_dataset=dataset_samsum_pt["train"],         # training dataset
    eval_dataset=dataset_samsum_pt["validation"],     # evaluation dataset
)



c:\Users\priks\anaconda3\envs\text-summarizer\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
# Start training
trainer.train()

  2%|▏         | 460/29440 [01:44<1:39:29,  4.85it/s]

{'eval_loss': 1.9598251581192017, 'eval_runtime': 3.9133, 'eval_samples_per_second': 209.032, 'eval_steps_per_second': 6.644, 'epoch': 1.0}


  2%|▏         | 500/29440 [01:53<1:42:00,  4.73it/s] 

{'loss': 2.4944, 'grad_norm': 1.68755042552948, 'learning_rate': 1.9960000000000002e-05, 'epoch': 1.09}


                                                     
  3%|▎         | 921/29440 [03:27<1:27:56,  5.41it/s]

...

                                                       
100%|██████████| 29440/29440 [2:24:09<00:00,  3.16it/s]

{'eval_loss': 1.6507415771484375, 'eval_runtime': 4.0337, 'eval_samples_per_second': 202.79, 'eval_steps_per_second': 6.446, 'epoch': 63.93}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
100%|██████████| 29440/29440 [2:24:10<00:00,  3.40it/s]

{'train_runtime': 8650.2567, 'train_samples_per_second': 108.997, 'train_steps_per_second': 3.403, 'train_loss': 1.7147044596464738, 'epoch': 63.93}


TrainOutput(global_step=29440, training_loss=1.7147044596464738, metrics={'train_runtime': 8650.2567, 'train_samples_per_second': 108.997, 'train_steps_per_second': 3.403, 'train_loss': 1.7147044596464738, 'epoch': 63.93})

In [17]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]




training_args.bin: 100%|██████████| 4.86k/4.86k [00:01<00:00, 4.48kB/s]
spiece.model: 100%|██████████| 792k/792k [00:02<00:00, 345kB/s] 4MB/s]
model.safetensors: 100%|██████████| 242M/242M [01:13<00:00, 3.29MB/s] 

Upload 3 LFS files: 100%|██████████| 3/3 [01:15<00:00, 25.07s/it]


CommitInfo(commit_url='https://huggingface.co/Prikshit7766/t5-small-samsum/commit/b1fc10c541454cb9ca44558042f8a2d1fe2ce8b5', commit_message='End of training', commit_description='', oid='b1fc10c541454cb9ca44558042f8a2d1fe2ce8b5', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [14]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

C:\Users\priks\AppData\Local\Temp\ipykernel_4636\2696170338.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
c:\Users\priks\anaconda3\envs\text-summarizer\lib\site-packages\datasets\load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [8]:
final_test_data = concatenate_datasets([test_dataset, validation_dataset])

In [9]:
final_test_data

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 1637
})

In [10]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Prikshit7766/t5-small-samsum")
model = AutoModelForSeq2SeqLM.from_pretrained("Prikshit7766/t5-small-samsum").to(device)

In [15]:
score = calculate_metric_on_test_ds(
    final_test_data, rouge_metric, model, tokenizer, batch_size = 32, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

rouge_dict

100%|██████████| 52/52 [30:08<00:00, 34.78s/it]


{'rouge1': 0.022735986144018477,
 'rouge2': 0.00038145898148848575,
 'rougeL': 0.022369386115261396,
 'rougeLsum': 0.022368230911790993}

In [17]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset["test"][0]["dialogue"]

reference = dataset["test"][0]["summary"]

pipe = pipeline("summarization", model="t5-small-samsum",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Betty's number is Lemme. Amanda can't find it. Larry called Betty last time they were at the park together.
